In [50]:
import pandas as pd
import numpy as np
import scipy
import random
import re
import datetime
import time

%matplotlib inline

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler
import sklearn.tree
import sklearn.linear_model
import sklearn.ensemble
import xgboost as xgboost
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
import imblearn

In [6]:
raw_df = pd.read_csv('unscrambl_data.csv')

In [7]:
raw_df = raw_df.drop_duplicates()

In [8]:
raw_df.columns = map(str.lower, raw_df.columns)
raw_df.columns = raw_df.columns.str.replace(' ', '_')

In [9]:
raw_df['name'] = raw_df['cardholder_last_name'] + '.' + raw_df['cardholder_first_initial']

In [10]:
raw_df['id'] = raw_df[['name']].sum(axis=1).map(hash)

In [11]:
raw_df['txn_id'] = raw_df.index + 1

In [12]:
raw_df.head()

,year-month,agency_number,agency_name,cardholder_last_name,cardholder_first_initial,description,amount,vendor,transaction_date,posted_date,merchant_category_code_(mcc),name,id,txn_id
0,201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,GENERAL PURCHASE,890.00,NACAS,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,CHARITABLE AND SOCIAL SERVICE ORGANIZATIONS,Mason.C,-6567662927092640105,1
1,201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,ROOM CHARGES,368.96,SHERATON HOTEL,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,SHERATON,Mason.C,-6567662927092640105,2
2,201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,J,GENERAL PURCHASE,165.82,SEARS.COM 9300,07/29/2013 12:00:00 AM,07/31/2013 12:00:00 AM,DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE...,Massey.J,-5512440702522637118,3
3,201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,T,GENERAL PURCHASE,96.39,WAL-MART #0137,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,"GROCERY STORES,AND SUPERMARKETS",Massey.T,-3208616730841106270,4
4,201307,1000,OKLAHOMA STATE UNIVERSITY,Mauro-Herrera,M,HAMMERMILL COPY PLUS COPY EA,125.96,STAPLES DIRECT,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,"STATIONERY, OFFICE SUPPLIES, PRINTING AND WRIT...",Mauro-Herrera.M,-5887776815949727836,5


In [13]:
transactions_df = pd.DataFrame().assign(TRANSACTION_ID=raw_df['txn_id'], TX_DATETIME=raw_df['transaction_date'], CUSTOMER_ID=raw_df['id'], TX_AMOUNT=raw_df['amount'])

transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TX_AMOUNT
0,1,07/30/2013 12:00:00 AM,-6567662927092640105,890.00
1,2,07/30/2013 12:00:00 AM,-6567662927092640105,368.96
2,3,07/29/2013 12:00:00 AM,-5512440702522637118,165.82
3,4,07/30/2013 12:00:00 AM,-3208616730841106270,96.39
4,5,07/30/2013 12:00:00 AM,-5887776815949727836,125.96


In [14]:
transactions_df['TX_AMOUNT'].describe(percentiles=[0.998])

count    4.274100e+05
mean     4.310604e+02
std      5.347359e+03
min     -4.286304e+04
50%      1.056500e+02
99.8%    1.274319e+04
max      1.903858e+06
Name: TX_AMOUNT, dtype: float64

In [15]:
start_date = '2013-04-17'

In [16]:
transactions_df = transactions_df.reindex(columns = transactions_df.columns.tolist() + ['TX_TIME_SECONDS', 'TX_TIME_DAYS', 'TX_FRAUD', 'TX_FRAUD_SCENARIO'])
transactions_df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,1,07/30/2013 12:00:00 AM,-6567662927092640105,890.00,NaN,NaN,NaN,NaN
1,2,07/30/2013 12:00:00 AM,-6567662927092640105,368.96,NaN,NaN,NaN,NaN
2,3,07/29/2013 12:00:00 AM,-5512440702522637118,165.82,NaN,NaN,NaN,NaN
3,4,07/30/2013 12:00:00 AM,-3208616730841106270,96.39,NaN,NaN,NaN,NaN
4,5,07/30/2013 12:00:00 AM,-5887776815949727836,125.96,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
442453,442454,06/20/2014 12:00:00 AM,-2839793698184850518,992.00,NaN,NaN,NaN,NaN
442454,442455,06/20/2014 12:00:00 AM,-2839793698184850518,2415.00,NaN,NaN,NaN,NaN
442455,442456,06/20/2014 12:00:00 AM,-2839793698184850518,177.50,NaN,NaN,NaN,NaN
442456,442457,06/19/2014 12:00:00 AM,-2839793698184850518,32457.00,NaN,NaN,NaN,NaN


In [17]:
transactions_df['TX_STARTTIME'] = pd.to_datetime(start_date)

In [18]:
transactions_df['TX_TIME_SECONDS'] = abs(pd.to_datetime(transactions_df['TX_DATETIME']) - pd.to_datetime(transactions_df['TX_STARTTIME'])).dt.total_seconds()

In [19]:
transactions_df['TX_TIME_DAYS'] = round(transactions_df['TX_TIME_SECONDS'] / 86400)

In [20]:
transactions_df['TX_TIME_DAYS'] = transactions_df['TX_TIME_DAYS'].astype(int)

In [21]:
def add_frauds(transactions_df):
    
    # By default, all transactions are genuine
    transactions_df['TX_FRAUD']=0
    transactions_df['TX_FRAUD_SCENARIO']=0
    
    # Scenario 1
    transactions_df.loc[transactions_df.TX_AMOUNT>12700, 'TX_FRAUD']=1
    transactions_df.loc[transactions_df.TX_AMOUNT>12700, 'TX_FRAUD_SCENARIO']=1
    nb_frauds_scenario_1=transactions_df.TX_FRAUD.sum()
    print("Number of frauds from scenario 1: "+str(nb_frauds_scenario_1))
    
    # Scenario 3
    for day in range(transactions_df.TX_TIME_DAYS.max()):
        
        compromised_customers = transactions_df.CUSTOMER_ID.sample(n=3, random_state=day).values
        
        compromised_transactions=transactions_df[(transactions_df.TX_TIME_DAYS>=day) & 
                                                    (transactions_df.TX_TIME_DAYS<day+14) & 
                                                    (transactions_df.CUSTOMER_ID.isin(compromised_customers))]
        
        nb_compromised_transactions=len(compromised_transactions)
        
        
        random.seed(day)
        index_fauds = random.sample(list(compromised_transactions.index.values),k=int(nb_compromised_transactions/3))
        
        transactions_df.loc[index_fauds,'TX_AMOUNT']=transactions_df.loc[index_fauds,'TX_AMOUNT']*5
        transactions_df.loc[index_fauds,'TX_FRAUD']=1
        transactions_df.loc[index_fauds,'TX_FRAUD_SCENARIO']=3
        
                             
    nb_frauds_scenario_3=transactions_df.TX_FRAUD.sum()-nb_frauds_scenario_1
    print("Number of frauds from scenario 3: "+str(nb_frauds_scenario_3))
    
    return transactions_df                 

In [22]:
transactions_df = add_frauds(transactions_df)

Number of frauds from scenario 1: 860
Number of frauds from scenario 3: 10236


In [23]:
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_STARTTIME
0,1,07/30/2013 12:00:00 AM,-6567662927092640105,890.00,8985600.0,104,0,0,2013-04-17
1,2,07/30/2013 12:00:00 AM,-6567662927092640105,368.96,8985600.0,104,0,0,2013-04-17
2,3,07/29/2013 12:00:00 AM,-5512440702522637118,165.82,8899200.0,103,0,0,2013-04-17
3,4,07/30/2013 12:00:00 AM,-3208616730841106270,96.39,8985600.0,104,0,0,2013-04-17
4,5,07/30/2013 12:00:00 AM,-5887776815949727836,125.96,8985600.0,104,0,0,2013-04-17


In [24]:
transactions_df['TX_DATETIME'] = pd.to_datetime(transactions_df['TX_DATETIME'])

In [25]:
def is_weekend(tx_datetime):
    
    # Transform date into weekday (0 is Monday, 6 is Sunday)
    weekday = tx_datetime.weekday()
    # Binary value: 0 if weekday, 1 if weekend
    is_weekend = weekday>=5
    
    return int(is_weekend)

In [26]:
transactions_df['TX_DURING_WEEKEND']=transactions_df.TX_DATETIME.apply(is_weekend)

In [27]:
def is_night(tx_datetime):
    
    # Get the hour of the transaction
    tx_hour = tx_datetime.hour
    # Binary value: 1 if hour less than 6, and 0 otherwise
    is_night = tx_hour<=6
    
    return int(is_night)

In [28]:
transactions_df['TX_DURING_NIGHT']=transactions_df.TX_DATETIME.apply(is_night)

In [29]:
def get_customer_spending_behaviour_features(customer_transactions, windows_size_in_days=[1,7,30]):
    
    # Let us first order transactions chronologically
    customer_transactions=customer_transactions.sort_values('TX_DATETIME')
    
    # The transaction date and time is set as the index, which will allow the use of the rolling function 
    customer_transactions.index=customer_transactions.TX_DATETIME
    
    # For each window size
    for window_size in windows_size_in_days:
        
        # Compute the sum of the transaction amounts and the number of transactions for the given window size
        SUM_AMOUNT_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').sum()
        NB_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').count()
    
        # Compute the average transaction amount for the given window size
        # NB_TX_WINDOW is always >0 since current transaction is always included
        AVG_AMOUNT_TX_WINDOW=SUM_AMOUNT_TX_WINDOW/NB_TX_WINDOW
    
        # Save feature values
        customer_transactions['CUSTOMER_ID_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        customer_transactions['CUSTOMER_ID_AVG_AMOUNT_'+str(window_size)+'DAY_WINDOW']=list(AVG_AMOUNT_TX_WINDOW)
    
    # Reindex according to transaction IDs
    customer_transactions.index=customer_transactions.TRANSACTION_ID
        
    # And return the dataframe with the new features
    return customer_transactions

In [30]:
transactions_df=transactions_df.groupby('CUSTOMER_ID').apply(lambda x: get_customer_spending_behaviour_features(x, windows_size_in_days=[1,7,30]))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)

In [31]:
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_STARTTIME,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW
0,31365,2013-04-17,-3682331964498518996,-34.82,0.0,0,0,0,2013-04-17,0,1,1.0,-34.82,1.0,-34.82,1.0,-34.82
1,147049,2013-04-30,-2407875687817207778,-7.00,1123200.0,13,0,0,2013-04-17,0,1,1.0,-7.00,1.0,-7.00,1.0,-7.00
2,9277,2013-05-16,-1621987688196298767,15120.00,2505600.0,29,1,1,2013-04-17,0,1,1.0,15120.00,1.0,15120.00,1.0,15120.00
3,1902,2013-05-21,4943060952955968653,-2.30,2937600.0,34,0,0,2013-04-17,0,1,1.0,-2.30,1.0,-2.30,1.0,-2.30
4,3488,2013-05-22,-3655169493295592264,-308.00,3024000.0,35,0,0,2013-04-17,0,1,1.0,-308.00,1.0,-308.00,1.0,-308.00


In [34]:
train_df, test_df = train_test_split(transactions_df, test_size=0.2)

In [35]:
train_df.shape

(341928, 17)

In [37]:
train_df[train_df['TX_FRAUD']==1].shape

(8844, 17)

In [38]:
test_df.shape

(85482, 17)

In [39]:
test_df[test_df['TX_FRAUD']==1].shape

(2252, 17)

In [40]:
output_feature="TX_FRAUD"

input_features=['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW']

In [44]:
# We first create a decision tree object. We will limit its depth to 2 for interpretability, 
# and set the random state to zero for reproducibility
classifier = sklearn.tree.DecisionTreeClassifier(max_depth = 2, random_state=0)

model_and_predictions_dictionary = fit_model_and_get_predictions(classifier, train_df, test_df, 
                                                                 input_features, output_feature,
                                                                 scale=False)

In [45]:
test_df['TX_FRAUD_PREDICTED']=model_and_predictions_dictionary['predictions_test']
test_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_STARTTIME,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TX_FRAUD_PREDICTED
272071,301851,2014-02-20,-5335962033181144324,-87.78,26697600.0,309,0,0,2013-04-17,0,1,1.0,-87.780000,1.0,-87.780000,4.0,63.450000,0.012501
62752,69811,2013-08-20,4940518227831745591,15.00,10800000.0,125,0,0,2013-04-17,0,1,2.0,114.700000,6.0,87.233333,11.0,99.926364,0.012501
211452,90280,2014-01-02,-8780777729231570830,315.94,22464000.0,260,0,0,2013-04-17,0,1,1.0,315.940000,4.0,284.807500,20.0,275.296000,0.012501
285930,337646,2014-03-05,1702124009874348155,178.50,27820800.0,322,0,0,2013-04-17,0,1,3.0,784.500000,3.0,784.500000,29.0,1858.573793,0.012501
95538,216799,2013-09-16,-3122693515629740204,118.89,13132800.0,152,0,0,2013-04-17,0,1,3.0,531.796667,5.0,362.076000,10.0,212.657000,0.012501


In [52]:
def fit_model_and_get_predictions(classifier, train_df, test_df, 
                                  input_features, output_feature="TX_FRAUD",scale=True):
    
    # We first train the classifier using the `fit` method, and pass as arguments the input and output features
    start_time=time.time()
    classifier.fit(train_df[input_features], train_df[output_feature])
    training_execution_time=time.time()-start_time

    # We then get the predictions on the training and test data using the `predict_proba` method
    # The predictions are returned as a numpy array, that provides the probability of fraud for each transaction 
    start_time=time.time()
    predictions_test=classifier.predict_proba(test_df[input_features])[:,1]
    prediction_execution_time=time.time()-start_time
    
    predictions_train=classifier.predict_proba(train_df[input_features])[:,1]

    # The result is returned as a dictionary containing the fitted models, 
    # and the predictions on the training and test sets
    model_and_predictions_dictionary = {'classifier': classifier,
                                        'predictions_test': predictions_test,
                                        'predictions_train': predictions_train,
                                        'training_execution_time': training_execution_time,
                                        'prediction_execution_time': prediction_execution_time
                                       }
    
    return model_and_predictions_dictionary

In [47]:
def performance_assessment(predictions_df, output_feature='TX_FRAUD', 
                           prediction_feature='predictions', rounded=True):
    
    AUC_ROC = metrics.roc_auc_score(predictions_df[output_feature], predictions_df[prediction_feature])
    AP = metrics.average_precision_score(predictions_df[output_feature], predictions_df[prediction_feature])
    
    performances = pd.DataFrame([[AUC_ROC, AP]], 
                           columns=['AUC ROC','Average precision'])
        
    if rounded:
        performances = performances.round(3)
    
    return performances

In [48]:
predictions_df=test_df
predictions_df['predictions']=model_and_predictions_dictionary['predictions_test']
    
performance_assessment(predictions_df)

,AUC ROC,Average precision
0,0.758,0.377


In [53]:
classifiers_dictionary={'Logistic regression':sklearn.linear_model.LogisticRegression(random_state=0), 
                        'Decision tree with depth of two':sklearn.tree.DecisionTreeClassifier(max_depth=2,random_state=0), 
                        'Decision tree - unlimited depth':sklearn.tree.DecisionTreeClassifier(random_state=0), 
                        'Random forest':sklearn.ensemble.RandomForestClassifier(random_state=0,n_jobs=-1),
                        'XGBoost':xgboost.XGBClassifier(random_state=0,n_jobs=-1),
                       }

fitted_models_and_predictions_dictionary={}

for classifier_name in classifiers_dictionary:
    
    model_and_predictions = fit_model_and_get_predictions(classifiers_dictionary[classifier_name], train_df, test_df, 
                                                                                  input_features=input_features,
                                                                                output_feature=output_feature)
    fitted_models_and_predictions_dictionary[classifier_name]=model_and_predictions

In [58]:
def performance_assessment_model_collection(fitted_models_and_predictions_dictionary, 
                                            transactions_df, 
                                            type_set='test'):

    performances=pd.DataFrame() 
    
    for classifier_name, model_and_predictions in fitted_models_and_predictions_dictionary.items():
    
        predictions_df=transactions_df
            
        predictions_df['predictions']=model_and_predictions['predictions_'+type_set]
        
        performances_model=performance_assessment(predictions_df, output_feature='TX_FRAUD', 
                                                   prediction_feature='predictions')
        performances_model.index=[classifier_name]
        
        performances=performances.append(performances_model)
        
    return performances

In [59]:
# performances on test set
df_performances=performance_assessment_model_collection(fitted_models_and_predictions_dictionary, test_df, 
                                                        type_set='test')
df_performances

,AUC ROC,Average precision
Logistic regression,0.839,0.389
Decision tree with depth of two,0.758,0.377
Decision tree - unlimited depth,0.798,0.380
Random forest,0.925,0.726
XGBoost,0.949,0.752
